In [ ]:
import random
import sys
import tensorflow as tf
from dataset_utils import _dataset_exists, _get_filenames_and_classes, write_label_file, _convert_dataset

## python create_tfrecord.py  --tfrecord_filename=200label --dataset_dir=/dataset-imagenet/
                           ## --num_shards=21 --validation_size=0.2 --random_seed=1
'''
#====================================================DEFINE YOUR ARGUMENTS=======================================================================
flags = tf.app.flags
#State your dataset directory
flags.DEFINE_string('dataset_dir', None, 'String: Your dataset directory')
# The number of images in the validation set. You would have to know the total number of examples in advance. This is essentially your evaluation dataset.
# The number of shards to split the dataset into
flags.DEFINE_integer('num_shards', 2, 'Int: Number of shards to split the TFRecord files')

# Seed for repeatability.
flags.DEFINE_integer('random_seed', 0, 'Int: Random seed to use for repeatability.')

#Output filename for the naming the TFRecord file
flags.DEFINE_string('tfrecord_filename', None, 'String: The output filename to name your TFRecord file')
FLAGS = flags.FLAGS
'''

tfrecord_filename='smotfrecord'
dataset_dir='/home/aewin/work/anaconda3/code/V001/smodata/'
#datasets_dir = '/home/aewin/work/anaconda3/code/V001/smorking'
num_shards=2
validation_size=0.2 
random_seed=1

def main():
    '''
    #==============================================================CHECKS==========================================================================
    #Check if there is a tfrecord_filename entered
    if not FLAGS.tfrecord_filename:
        raise ValueError('tfrecord_filename is empty. Please state a tfrecord_filename argument.')

    #Check if there is a dataset directory entered
    if not FLAGS.dataset_dir:
        raise ValueError('dataset_dir is empty. Please state a dataset_dir argument.')

    #If the TFRecord files already exist in the directory, then exit without creating the files again
    if _dataset_exists(dataset_dir = FLAGS.dataset_dir, _NUM_SHARDS = FLAGS.num_shards, output_filename = FLAGS.tfrecord_filename):
        print 'Dataset files already exist. Exiting without re-creating them.'
        return None
    #==============================================================END OF CHECKS===================================================================
    '''
    #Get a list of photo_filenames like ['123.jpg', '456.jpg'...] and a list of sorted class names from parsing the subdirectories.
    photo_filenames, class_names = _get_filenames_and_classes(dataset_dir)
    print(class_names,len(class_names))
    print(type(class_names))
    print ('\n Checking...0')
    #Refer each of the class name to a specific integer number for predictions later
    class_id = [int(i) for i in range(len(class_names))]
    print(class_id,len(class_id))
    print(type(class_id))
    print(class_id)
    
    class_names_to_ids = dict(zip(class_names, class_id))
    #class_names_to_ids = dict(zip(class_names, range(len(class_names))))
    print(class_names_to_ids)
    #Find the number of validation examples we need
    exit(1)
    num_validation = int(validation_size * len(photo_filenames))
    print ('\n Checking')
    print (class_names_to_ids)
    print (len(class_names_to_ids))
    
    
    sys.stdout.write('\n num_validation: %d, num class number %d \n' % ( num_validation, len(class_names) ))
    sys.stdout.flush()

    # Divide the training datasets into train and test:
    random.seed(random_seed)
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[num_validation:]
    validation_filenames = photo_filenames[:num_validation]
    print ('----------------')
    print (training_filenames)
    print (tfrecord_filename)
    print (num_shards)
    print ('----------------')
    # First, convert the training and validation sets.
    _convert_dataset('train', training_filenames, class_names_to_ids,
                     dataset_dir = dataset_dir, tfrecord_filename = tfrecord_filename, _NUM_SHARDS = num_shards)
    
    
    _convert_dataset('validation', validation_filenames, class_names_to_ids,
                     dataset_dir = dataset_dir, tfrecord_filename = tfrecord_filename, _NUM_SHARDS = num_shards)

    # Finally, write the labels file:
    labels_to_class_names = dict(zip(class_id, class_names))
    #labels_to_class_names = dict(zip(range(len(class_names)), class_names))
    write_label_file(labels_to_class_names, dataset_dir)
    
    print ('\nFinished converting the %s dataset!' % (tfrecord_filename))

if __name__ == "__main__":
    main()
   

In [ ]:

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import slim
import os
import time
file_pattern = 'smotfrecord_train_%s.tfrecord'

def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 
    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = file_pattern

    #Count the total number of examples in all of these shard
    num_samples = 0
    '''
    file_pattern_for_counting = '200label_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    print(file_pattern_for_counting)
    '''
    tfrecords_to_count = [os.path.join(dataset_dir,file) for file in os.listdir(dataset_dir)]
         
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    
    print(tfrecords_to_count)
    
    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name
    
    print(tfrecords_to_count)
    
    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = tfrecords_to_count,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)
    
    
    print(dataset.data_sources)
    return dataset


get_split('train', datasets_data_dir, file_pattern=file_pattern)
#file_pattern = 'smotfrecord_validation_%s.tfrecord'
#get_split('validation', datasets_data_dir, file_pattern=file_pattern)

In [ ]:
from datasets import flowers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import slim

with tf.Graph().as_default(): 
    dataset = get_split('train', dataset_dir)
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32, common_queue_min=1)
    image, label = data_provider.get(['image', 'label'])
    
    